<a href="https://colab.research.google.com/github/SahilCarterr/FinetuningLLM/blob/main/gemma_7B_finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip -q install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-cola

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
alpaca_prompt = """Below is an question which corresponds to topic. Write a topic that appropriate for the question.

### question:
{}

### topic:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    question = examples["question"]
    topic = examples["topic"]
    texts = []
    for question, topic in zip(question, topic):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(question, topic) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

# from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
import pandas as pd
df = pd.read_csv('/content/data_withoutLENE.csv')
unique_topics = df['topic'].unique()
topic_mapping = {topic: i for i, topic in enumerate(unique_topics)}

# # Apply the mapping to the 'topic' column
# df['topic'] = df['topic'].map(topic_mapping)

# # Display the updated DataFrame and the mapping of unique strings to numbers
# print("Mapping of unique strings to numbers:", topic_mapping)
import pandas as pd
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Save the train and test datasets to new CSV files
train_data.to_csv('train.csv', index=False)
test_data.to_csv('test.csv', index=False)

print("Data has been split and saved to train.csv and test.csv.")


Data has been split and saved to train.csv and test.csv.


In [ ]:
!pip -q install pyarrow datasets

In [ ]:
from datasets import load_dataset
data_files = {"train": "/content/train.csv", "test": "/content/test.csv"}
dataset = load_dataset("csv", data_files=data_files)
#select train split of dataset
#dataset = dataset["train"]

In [ ]:
dataset = dataset["train"]
dataset

Dataset({
    features: ['question', 'topic'],
    num_rows: 1274
})

In [ ]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1274 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs=1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/1274 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,274 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 159
 "-____-"     Number of trainable parameters = 50,003,968


Step,Training Loss
1,4.486300
2,4.107500
3,4.064300
4,3.368400
5,2.714300
6,2.439600
7,2.308900
8,1.870900
9,1.666200
10,1.525100


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        dataset["question"][2],#question
        "", # topic - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 20, use_cache = True)
a=tokenizer.batch_decode(outputs)

In [ ]:
import re
pattern = r'### topic:\n(.+)<'

# Search for the pattern in the text
match = re.search(pattern, a[0])

if match:
    topic = match.group(1)
    print(f"{topic}")
    if(topic==dataset["topic"][2]):
      print("Correct")
    else:
      print("Incorrect")
else:
    print("No topic found.")

Finance
Correct


In [ ]:
dataset_test= dataset["test"]

In [ ]:
dataset_test['question'][0]

'Guess the name of the girl'

In [ ]:
len(dataset_test['question'])

319

In [ ]:
dataset_test

Dataset({
    features: ['question', 'topic'],
    num_rows: 319
})

In [ ]:
c=0
for i in range(len(dataset_test['question'])):
  # alpaca_prompt = Copied from above
  FastLanguageModel.for_inference(model) # Enable native 2x faster inference
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          dataset_test['question'][i],#question
          "", # topic - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")
  outputs = model.generate(**inputs, max_new_tokens = 20, use_cache = True)
  a=tokenizer.batch_decode(outputs)
  import re
  pattern = r'### topic:\n(.+)<'

  # Search for the pattern in the text
  match = re.search(pattern, a[0])

  if match:
      topic = match.group(1)
      #print(f"{topic}")
      if(topic==dataset_test["topic"][i]):
        #print("Correct")
        c+=1
      else:
        print("Incorrect")

In [ ]:
c

246

In [ ]:
print(f"Accuracy is {c/len(dataset_test['question'])}")

Accuracy is 0.7711598746081505


In [ ]:
from collections import defaultdict

classes = ['Dating and Relationships', 'Random', 'Movies and TV', 'Music', 'Technology and Internet', 'Philosophy and Religion', 'technology and Internet', 'Law and Order', 'Health and Wellness', 'Food and Drinks', 'Fashion and Makeup', 'health and Wellness', 'Gaming', 'Sports', 'fashion and Makeup', 'sports', 'Anime', 'anime', 'Finance', 'Travel and Geography', 'Pets and Animals', 'gaming', 'random', 'music']

# Convert words to class indices
def word_to_class_index(word):
    return classes.index(word)

y_true = [word_to_class_index(word) for word in y_true_words]
y_pred = [word_to_class_index(word) for word in y_pred_words]

# Initialize dictionaries to store counts for each class
class_counts = defaultdict(lambda: {'TP': 0, 'FP': 0, 'FN': 0})

# Calculate TP, FP, FN for each class
for true, pred in zip(y_true, y_pred):
    if true == pred:
        class_counts[true]['TP'] += 1
    else:
        class_counts[true]['FN'] += 1
        class_counts[pred]['FP'] += 1

# Calculate Precision and Recall for each class
class_metrics = {}
for cls in classes:
    TP = class_counts[classes.index(cls)]['TP']
    FP = class_counts[classes.index(cls)]['FP']
    FN = class_counts[classes.index(cls)]['FN']

    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0

    class_metrics[cls] = {'Precision': precision, 'Recall': recall}

# Print results
for cls, metrics in class_metrics.items():
    print(f'Class: {cls}')
    print(f'Precision: {metrics["Precision"]:.2f}')
    print(f'Recall: {metrics["Recall"]:.2f}')
    print()

Class: Dating and Relationships
Precision: 0.33
Recall: 0.2

Class: Random
Precision: 0.7
Recall: 0.83

Class: Movies and TV
Precision: 0.72
Recall: 0.81

Class: Music
Precision: 0.82
Recall: 0.9

Class: Technology and Internet
Precision: 0.33
Recall: 0.18

Class: Philosophy and Religion
Precision: 0.0
Recall: 0.0

Class: technology and Internet
Precision: 0.0
Recall: 0.0

Class: Law and Order
Precision: 0.79
Recall: 0.85

Class: Health and Wellness
Precision: 0.31
Recall: 0.62

Class: Food and Drinks
Precision: 0.89
Recall: 0.62

Class: Fashion and Makeup
Precision: 0.89
Recall: 0.73

Class: health and Wellness
Precision: 0.0
Recall: 0.0

Class: Gaming
Precision: 1.0
Recall: 0.82

Class: Sports
Precision: 0.9
Recall: 0.95

Class: fashion and Makeup
Precision: 0.0
Recall: 0.0

Class: sports
Precision: 0.0
Recall: 0.0

Class: Anime
Precision: 0.82
Recall: 0.88

Class: anime
Precision: 0.0
Recall: 0.0

Class: Finance
Precision: 1.0
Recall: 0.88

Class: Travel and Geography
Precision: 0.8